<a href="https://colab.research.google.com/github/higor-gomes93/ona_research/blob/main/AIHR_ONA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1) Libraries

In [44]:
# Libraries
!pip install pyvis dash dash_cytoscape
import pandas as pd
import random as rd
import numpy as np
import json
import pyvis as pv
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
from matplotlib.pyplot import figure
from IPython.core.display import display, HTML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Connection with Google Drive and Google Sheets
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import drive 
from google.auth import default
drive.mount('/content/gdrive')
from gspread_dataframe import set_with_dataframe
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/gdrive


### 2) Importing the Dataset

In [3]:
# Getting the raw data
with open("gdrive/My Drive/Área de Trabalho/Documentos/Datasets/starwars-full-interactions-allCharacters.json") as json_data:
    data = json.load(json_data)

In [4]:
# Breaking the data into 2 datasets: nodes and edges
dataset_nodes = pd.DataFrame.from_dict(data['nodes'])
dataset_edges = pd.DataFrame.from_dict(data['links'])

In [14]:
# Creating a unique dataset
# New copy
dataset = dataset_edges.copy()
# Sources
dataset['source_name'] = dataset['source'].apply(lambda x: dataset_nodes.iloc[x]['name'])
dataset['source_size'] = dataset['source'].apply(lambda x: dataset_nodes.iloc[x]['value'])
dataset['source_colour'] = dataset['source'].apply(lambda x: dataset_nodes.iloc[x]['colour'])
# Targets
dataset['target_name'] = dataset['target'].apply(lambda x: dataset_nodes.iloc[x]['name'])
dataset['target_size'] = dataset['target'].apply(lambda x: dataset_nodes.iloc[x]['value'])
dataset['target_colour'] = dataset['target'].apply(lambda x: dataset_nodes.iloc[x]['colour'])
# Droping some columns
dataset = dataset.drop(['source', 'target'], axis = 1)
# Renaming
dataset.columns = ['edge', 'source_name', 'source_size', 'source_colour', 'target_name', 'target_size', 'target_colour']
dataset = dataset[['source_name', 'source_size', 'source_colour', 'target_name', 'target_size', 'target_colour', 'edge']]

### 3) Graph Visualization

In [41]:
# Creating the graph object
graph = Network(height='1000px', width='100%', bgcolor='#111111', font_color='white', directed = False)
graph.barnes_hut(gravity = -80000, central_gravity=0.3, spring_length = 250, spring_strength=0.01, damping=0.09, overlap = 0)

In [42]:
# Adding the data
for i in range(len(dataset)):
    source = dataset.iloc[i]['source_name']
    source_group = dataset.iloc[i]['source_colour']
    source_size = int(dataset.iloc[i]['source_size'])
    target = dataset.iloc[i]['target_name']
    target_group = dataset.iloc[i]['target_colour']
    target_size = int(dataset.iloc[i]['target_size'])
    weight = int(dataset.iloc[i]['edge'])

    graph.add_node(source, title = source, group = source_group, size = source_size)
    graph.add_node(target, title = target, group = target_group, size = target_size)
    graph.add_edge(source, target, width = weight, color = "#282828")

In [45]:
# Ploting
graph.show('graph.html')
display(HTML('graph.html'))